# Урок 4. Парсинг HTML. XPath

## 1 задание. Написать приложение, которое собирает основные новости с сайтов news.mail.ru, lenta.ru, yandex-новости. Для парсинга использовать XPath. 

Структура данных должна содержать:
- название источника;
- наименование новости;
- ссылку на новость;
- дата публикации.

In [1]:
from lxml import html
from pprint import pprint
import requests
from pymongo import MongoClient

In [2]:
main_link_mail = 'https://news.mail.ru'
main_link_lenta = 'https://lenta.ru'
main_link_yandex = 'https://yandex.ru/news'

In [3]:
# Функция для получения DOM сайта. В качестве аргумента принимает ссылку
def get_dom(link):
    header = {'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36'}
    response = requests.get(link, headers=header)
    dom = html.fromstring(response.text)
    return dom

# Функция для очистки списка и отображения первого элемента. В качестве аргумента принимает список
def list_len(list):
    try:
        el = list[0].replace('\xa0', ' ')
        return el
    except:
        pass
# Функция конструктор ссылок. В качестве аргумента принимает основную ссылку и сокращенную    
def link_constr(main_link, link):
    if 'https://' not in link:
        link = main_link + link
    else:
        link
    return link

# Функция добавления документов в коллекции. В качестве аргументов принимает добавляемый словарь и коллекцию для добавления
def add_collection(news_dict, collection):
    for new in news_dict:
        count = 0
        if collection.count_documents({'title': new['title']}) == 0:
            collection.insert_one(new)
            count += 1
    print(f'В коллекцию {collection.name} добавлено {count} документов. Всего документов в коллекции {collection.count_documents({})}')

In [4]:
# Сбор новостей с сайта lenta.ru

dom_lenta = get_dom(main_link_lenta)

section = dom_lenta.xpath("//section[@class='mg-grid__row mg-grid__row_gap_8 news-top-flexible-stories news-app__top']//div[@class='item'] | //section[@class='row b-top7-for-main js-top-seven']//div[@class='first-item']")

lenta_news =[]

for new in section:
    lenta_new = {}
    title = list_len(new.xpath("./a/text() | ./h2/a/text()"))

    link = list_len(new.xpath(".//@href | ./h2//@href"))
    link = link_constr(main_link_lenta, link)
    
    date = list_len(new.xpath(".//time/@datetime | .//time/@datetime"))
    source = main_link_lenta
    
    lenta_new['source'] = source
    lenta_new['title'] = title
    lenta_new['link'] = link
    lenta_new['date'] = date
    
    lenta_news.append(lenta_new)
    
pprint(lenta_news[0])

{'date': ' 19:29, 17 марта 2021',
 'link': 'https://lenta.ru/news/2021/03/17/crmiea/',
 'source': 'https://lenta.ru',
 'title': 'В Госдуме ответили на заявление Украины об обязанности Германии '
          'вернуть Крым'}


In [5]:
# Сбор новостей с сайта yandex.ru

dom_yandex = get_dom(main_link_yandex)

section = dom_yandex.xpath("//div[@class = 'mg-grid__row mg-grid__row_gap_8 news-top-flexible-stories news-app__top']") 

yandex_news =[]

for new in section:
    yandex_new = {}

    title = list_len(new.xpath(".//h2/text()"))
    link = list_len(new.xpath(".//a/@href"))
    link = link_constr(main_link_yandex, link)
    
    date = list_len(new.xpath(".//span[@class = 'mg-card-source__time']/text()"))
    source = list_len(new.xpath(".//span[@class = 'mg-card-source__source']/a/text()"))
    
    yandex_new['source'] = source
    yandex_new['title'] = title
    yandex_new['link'] = link
    yandex_new['date'] = date   
    
    yandex_news.append(yandex_new)
    
pprint(yandex_news[0])

{'date': '19:38',
 'link': 'https://yandex.ru/news/story/Bajden_zayavil_ob_otvetstvennosti_RF_zavmeshatelstvo_vvybory_vSSHA--3afd2e6a99e7af6392552cde07daf6e1?lang=ru&rubric=index&fan=1&stid=zCdsXO5uGpU2dXzg3fk4&t=1615999375&tt=true&persistent_id=135715037',
 'source': 'Известия',
 'title': 'Байден заявил об ответственности РФ за вмешательство в выборы в США'}


In [6]:
# Сбор новостей с сайта mail.ru

dom_mail = get_dom(main_link_mail)

section = dom_mail.xpath("//div[contains(@data-counter-id, '20268335')]//a[contains(@class, photo)]")

mail_news =[]

for new in section:
    mail_new = {}

    link = list_len(new.xpath(".//@href"))
    link = link_constr(main_link_mail, link)
    
    temp_dom = get_dom(link)
    
    title = list_len(temp_dom.xpath("//h1/text()"))
    date = list_len(temp_dom.xpath("//span[@class = 'note__text breadcrumbs__text js-ago']/@datetime"))
    source = list_len(temp_dom.xpath("//a[@class = 'link color_gray breadcrumbs__link']//span/text()"))
    
    mail_new['source'] = source
    mail_new['title'] = title
    mail_new['link'] = link
    mail_new['date'] = date   
    
    mail_news.append(mail_new)
    
pprint(mail_news[0])

{'date': '2021-03-17T17:26:12+03:00',
 'link': 'https://news.mail.ru/economics/45591121/',
 'source': 'Известия',
 'title': 'Счетная палата сочла риск топливного кризиса в России минимальным'}


## 2 задание. Сложить собранные данные в БД

In [7]:
# Подключаемся к MongoDB
client = MongoClient('127.0.0.1', 27017)

# Cоздаем указатель к нашей базе данных
db = client['news_database']

# Создаем коллекции
news_lenta = db.news_lenta
news_yandex = db.news_yandex
news_mail = db.news_mail

# Добавляем документы в соответствующие коллкции
add_collection(lenta_news, news_lenta)
add_collection(mail_news, news_mail)
add_collection(yandex_news, news_yandex)

В коллекцию news_lenta добавлено 0 документов. Всего документов в коллекции 10
В коллекцию news_mail добавлено 0 документов. Всего документов в коллекции 11
В коллекцию news_yandex добавлено 0 документов. Всего документов в коллекции 1
